In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim

import pandas as pd
import numpy as np

import tqdm
import matplotlib.pyplot as plt

import re

In [2]:
# Загружаем файлы и готовим некоторые списки

df = pd.read_csv('гороскопы мейла с фичами.csv', sep=';') # в этом файле есть координаты, но только по 2 первых слова прогноза

signs = ['Овен',	'Телец',	'Близнецы',	'Рак',	'Лев',	'Дева',	'Весы',	'Скорпион',	'Стрелец',	'Козерог',	'Водолей',	'Рыбы']
planets = {'Sun': 10,  'Mercury': 199, 'Venus': 299, 'Mars': 499, 'Jupiter': 599, 'Saturn': 699, 'Uranus': 799, 'Neptune': 899, 'Pluto': 999}
features = []
for k, v in planets.items():
  features += [k+'_x', k+'_y', k+'_z', k+'_vx', k+'_vy', k+'_vz', k+'_l', k+'_ry', k+'_rr']

In [4]:
df_orig = pd.read_csv('df_three.csv', sep=';') # это оригинальный файл с соскрапленными гороскопами

def conv_date(d):
  d = d.split()
  year = d[-1]
  day = d[0]
  if d[1] == 'января':
    month = '01'
  elif d[1] == 'февраля':
    month = '02'
  elif d[1] == 'марта':
    month = '03'
  elif d[1] == 'апреля':
    month = '04'
  elif d[1] == 'мая':
    month = '05'
  elif d[1] == 'июня':
    month = '06'
  elif d[1] == 'июля':
    month = '07'
  elif d[1] == 'августа':
    month = '08'
  elif d[1] == 'сентября':
    month = '09'
  elif d[1] == 'октября':
    month = '10'
  elif d[1] == 'ноября':
    month = '11'
  elif d[1] == 'декабря':
    month = '12'

  return '-'.join([year, month, day])

df_orig = df_orig[['date'] + signs].copy()
df_orig.fillna('0')
df_orig['date'] =  df_orig['date'].apply(lambda x: conv_date(x))
# df_orig['date_for_epoch'] =  df_orig['date'].apply(lambda x: Time(x).jd)

start_sign_cols = []
for sign in signs:
  start_sign_cols.append('start_' + sign)
  df_orig['start_' + sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()[0:5]).lower() if type(x) == str else x)
  df_orig[sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()).lower() if type(x) == str else x)

In [5]:
df[start_sign_cols] = df_orig[start_sign_cols]

In [6]:
df_orig.head(2)

,date,Овен,Телец,Близнецы,Рак,Лев,Дева,Весы,Скорпион,Стрелец,Козерог,Водолей,Рыбы,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы
0,2020-10-18,насыщенный интересный день за любые дела вы бе...,вам может быть нелегко отстоять свои интересы ...,тот самый день когда ваши поступки вызывают вс...,для самостоятельной работы это не самый подход...,вы без труда завоевываете симпатии новых знако...,первая половина дня проходит спокойно можно де...,благоприятный день для общения удается произве...,не спешите это не тот день когда можно за счит...,напряженный день будьте осторожны и внимательн...,с эмоциональной точки зрения день будет доволь...,утром стоит быть осторожнее поскольку в это вр...,здравый смысл и практичность помогают избежать...,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают
1,2020-10-17,будьте практичны этот день может положить нача...,день складывается удачно хотя порой вы сами се...,благоприятный день для любых встреч и перегово...,день подходит для решения любых вопросов в том...,вы способны на многое и сами это понимаете но ...,утро потребует осторожности в делах и сдержанн...,удачный день открываются новые возможности и в...,утро проходит очень плодотворно в это время вы...,вы решите многие проблемы если расстанетесь с ...,день складывается напряженно но влияние благоп...,неплохой день подходящий для любого общения вы...,день хоть и не лишен сложных моментов но все ж...,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен


In [7]:
df.head(2)

,Unnamed: 0,date,date_for_epoch,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,...,Jupiter_vz,Jupiter_l,Jupiter_ry,Jupiter_rr,Saturn_x,Saturn_y,Saturn_z,Saturn_vx,Saturn_vy,Saturn_vz,Saturn_l,Saturn_ry,Saturn_rr,Uranus_x,Uranus_y,Uranus_z,Uranus_vx,Uranus_vy,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr
0,0,2020-10-18,2459140.5,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.54520,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,...,-0.000161,0.029598,5.124750,0.015327,4.254547,-8.982140,-0.056410,0.011974,-0.012692,-0.000228,0.057403,9.938973,0.016597,14.631491,11.826922,-0.155757,0.005041,-0.012651,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819
1,1,2020-10-17,2459139.5,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.55172,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,...,-0.000161,0.029509,5.109411,0.015350,4.242709,-8.969385,-0.056182,0.011702,-0.012818,-0.000228,0.057307,9.922379,0.016592,14.626586,11.839634,-0.155800,0.004768,-0.012774,0.000043,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824


In [8]:
df = df.drop(['date',	'date_for_epoch'], axis=1)

In [9]:
df_orig = df_orig.dropna()

In [10]:
df_dommagii = pd.read_csv('df_three_dommagii.csv', sep=';')
for sign in signs:
  df_dommagii[sign] = df_dommagii[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()).lower() if type(x) == str else x)
data_part_dommagii = [d.split() for d in list(df_dommagii[signs].values.reshape(25332)) if type(d)==str]

In [11]:
df_hyrax = pd.read_csv('df_three_hyrax.csv', sep=';')
for sign in signs:
  df_hyrax[sign] = df_hyrax[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()).lower() if type(x) == str else x)
data_part_hyrax = [d.split() for d in list(df_hyrax[signs].values.reshape(48180)) if type(d)==str]

In [12]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [16]:
data_part = [d.split() for d in list(df_orig[signs].values.reshape(3708))]
model_50 = Word2Vec(data_part+data_part_dommagii+data_part_hyrax, min_count=1, size=50, workers=3, window =3, sg = 1)

In [17]:
model_50['день'], model_50.most_similar('день'), model_50.similar_by_vector(model_50['день'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython ker

(array([-0.15200637,  0.45711643, -0.4947711 , -0.05775643,  0.7853249 ,
        -0.14172612, -1.0089407 , -0.12738541, -0.7831307 , -0.26239896,
        -1.3062277 , -0.11031161,  0.7230398 ,  1.0391508 , -0.6661155 ,
         0.2415261 ,  0.7301659 ,  0.2590898 ,  0.12278003,  0.77527034,
        -0.72536355,  0.05690978,  1.0715315 , -0.9961927 ,  1.1010075 ,
        -0.28622454, -1.4190803 ,  0.88340133, -0.8050651 , -0.39381388,
         0.11880241,  0.300634  , -0.18001854, -1.210484  , -0.13961573,
        -0.14345734,  0.07465824, -2.2385645 ,  0.2065428 , -0.12872735,
        -0.4413517 , -0.7586338 , -0.48587292,  0.02392701,  0.39447287,
        -0.33206597, -0.6452487 , -0.13460226, -0.70199484, -1.3378814 ],
       dtype=float32),
 [('для', 0.6642758250236511),
  ('праздной', 0.6635351777076721),
  ('активный', 0.6629836559295654),
  ('благоприятный', 0.6535197496414185),
  ('этот', 0.649883508682251),
  ('непростым', 0.645966112613678),
  ('продуктивный', 0.64505547285079

In [18]:
model_50.save("word2vec_size_50_all_datasets.model")
# model_50 = Word2Vec.load("word2vec_size_50_all_datasets.model")

In [22]:
model_300 = Word2Vec(data_part+data_part_dommagii+data_part_hyrax, min_count=1, size=300, workers=3, window =3, sg = 1)
model_300.save("word2vec_size_300_all_datasets.model")

In [23]:
model_300['день'], model_300.most_similar('день'), model_300.similar_by_vector(model_300['день'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython ker

(array([-3.77254814e-01,  2.30618939e-01,  1.97889805e-02, -1.07127279e-02,
         7.95874149e-02, -1.38792410e-01, -4.81437385e-01,  4.23501134e-01,
        -5.20182192e-01, -1.56897053e-01, -4.75196242e-01,  1.00737512e-01,
         1.19763970e-01,  6.31079137e-01,  5.27446419e-02,  1.16041958e-01,
         1.16061956e-01, -1.74857080e-01,  7.89032653e-02,  3.79799664e-01,
        -4.19931442e-01, -3.10360223e-01, -7.87608549e-02, -5.93668044e-01,
        -4.14740801e-01, -2.69705236e-01, -7.63080299e-01,  4.93759751e-01,
        -1.74862236e-01,  6.14319704e-02, -2.38490198e-02,  5.84307671e-01,
        -3.78306478e-01, -6.07318342e-01, -4.02027994e-01,  2.53230799e-03,
        -4.16252129e-02, -3.95630598e-01, -1.84847170e-03, -2.35183001e-01,
        -2.35309064e-01, -1.43366307e-01,  2.05739781e-01,  1.25073522e-01,
         4.68565345e-01,  9.22370777e-02, -2.74542540e-01,  1.46043390e-01,
        -3.90069872e-01, -2.36543298e-01,  2.13410541e-01, -1.44725010e-01,
         7.4

In [24]:
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [30]:
data_part_5_vec_50 = [np.array(model_50[el[0:5]]).sum(axis=0) for el in data_part]
data_part_5_words_50 = [' '.join(el[0:5]) for el in data_part]

vector_length = 50
kv_50 = WordEmbeddingsKeyedVectors(vector_length)
kv_50.add(data_part_5_words_50, data_part_5_vec_50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [31]:
kv_50.similar_by_vector(np.random.uniform(size=50))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('непросто а порой и очень', 0.041132740676403046),
 ('коекакие трудности могут возникнуть но', 0.035038672387599945),
 ('могут возникать трудности но едва', 0.022460119798779488),
 ('отбросьте ложную скромность и постарайтесь', 0.02240026742219925),
 ('не помешают спокойствие и уединение', 0.020614270120859146),
 ('чаще обычного складываются удивительные и', 0.01954500749707222),
 ('начало дня будет нервным и', 0.01862482726573944),
 ('мелкие недоразумения и незначительные трудности', 0.01802467182278633),
 ('не исключены трудности и неприятные', 0.017790760844945908),
 ('может быть трудно сохранять самообладание', 0.017361287027597427)]

In [32]:
kv_50.save("WordEmbeddingsKeyedVectors_size_50.model")

Как загружать

In [33]:
# Как загружать

# kv_50 = WordEmbeddingsKeyedVectors.load("WordEmbeddingsKeyedVectors_size_50.model")

In [35]:
data_part_5_vec_300 = [np.array(model_300[el[0:5]]).sum(axis=0) for el in data_part]
data_part_5_words_300 = [' '.join(el[0:5]) for el in data_part]

vector_length = 300
kv_300 = WordEmbeddingsKeyedVectors(vector_length)
kv_300.add(data_part_5_words_300, data_part_5_vec_300)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [40]:
kv_300.similar_by_vector(np.random.uniform(size=300))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('начинается день напряженно трудности могут', 0.06032707542181015),
 ('появятся новые идеи они будут', 0.05627724900841713),
 ('стоит доверять своему внутреннему голосу', 0.05542787164449692),
 ('важно правильно оценивать свои силы', 0.05353400856256485),
 ('неплохой день коекакие трудности могут', 0.049757275730371475),
 ('у вас многое получается хорошо', 0.04664461314678192),
 ('у вас многое получается хорошо', 0.04664461314678192),
 ('спокойный день лучше всего посвятить', 0.04658500850200653),
 ('действуйте решительно у вас есть', 0.04571674019098282),
 ('отличный день у вас многое', 0.04559712111949921)]

In [41]:
kv_300.save("WordEmbeddingsKeyedVectors_size_300.model")